In [ ]:
import numpy as np

from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint
from scipy.optimize import Bounds
from scipy import optimize
import pandas as pd

#This function finds the configuration of the filament bundle delivering energy minimum
#as the function of dimensionless parameter q (in the paper zeta) and torque A.
#Boundary conditions are specified to be boundary conditions for one braid.
#The units are chosen such that:
#the bending modulus of  a single filament kappa = 1 
#cross link size a = 1.

#Function returns: 
#As the first output, the half-angle of the kink produced by a single braid
#in the state with minimal energy
#As the second output, the energy of the braid
#As the third output, the length of the filament in the braid
 


    
def solver2_cor(q, A): 
    
    N = 100
    b1 = np.zeros(N) - 1.5

    b2 = np.zeros(N) + 1.5

    bounds = []

    for i in range(N-2):
        bounds.append((-1.5, 1.5))

    bounds.append((0, 2))
    bounds.append((0.0001, 100))

   


    def separation_first_step(X):
        Ntot = len(X)
        n1 = int(Ntot / 2)
        X1 = X[:n1]
        X2 = X[n1:]
        return(X1,X2)
    
    def separation2(X):
        Nx = len(X)
        n = int(Nx/2 - 1)
        
          
        phi = X[Nx-2]
        dt = X[Nx - 1]**2
        
        inx1 = np.append([0 - phi],X[:n])
        x1 = np.append(inx1,[phi])
        inx2 = np.append([0 - phi],X[n:2*n])
        x2 = np.append(inx2,[phi])
        
        return(x1, x2, phi, dt)
    


    


    

        
    def integral_constraint_cor(X):
        x1, x2, phi, dt = separation2(X)
        L1 = np.sum(np.sin(x1))*dt -  np.cos(phi)
        L2 = np.sum(np.sin(x2))*dt -  np.cos(phi)
        
             
    
        
        L = L1 + L2
    
        return(L)

    
        
    def integral_constraint_2_cor(X):
       
        x1, x2, phi, dt = separation2(X)
        L1 = np.sum(np.cos(x1))*dt
        L2 = np.sum(np.cos(x2))*dt

        L = L1 - L2
        return(L)
 

    
    
    

    
    Y =  np.random.uniform(-0.1,0.1, N)
    Y[N - 1] = 0.1
    
    def energy_one(x, dt):
        
        dx = np.diff(x)
        E = np.sum(dx**2)  / (dt * 2) + q * dt * len(x) 
    
        
        return(E)
    
    def energy_double(x, dt):
        
        dx = np.diff(x)
        E = 2* np.sum(dx**2)  / (dt * 2) + q * dt * len(x) 
        return(E)
    
    
    def tot_en(X):
       
        x1, x2, phi, dt= separation2(X)
       
        E1 = energy_one(x1, dt)
        E2 = energy_double(x2, dt)
        
    
        return(E1 + E2  + A * (1 - np.cos(2*phi)) / 2)


    
    
    
    int_cons1 = NonlinearConstraint(integral_constraint_cor,0, 0 )  
    int_cons2 = NonlinearConstraint(integral_constraint_2_cor,0, 0 ) 
    
    n_trials = 5
    phi_val = np.zeros(n_trials)
    Y = np.zeros(N)
    dtval = np.zeros(n_trials)
    En_val = np.zeros(n_trials)
    div = np.zeros(n_trials)
    resX = np.zeros(len(Y)*n_trials).reshape(len(Y), n_trials)
    for i in range(n_trials):
        Y =  np.random.uniform(-1.0,1.0, N)
        Y[N-1] = 0.1
        print(i)
        res = minimize(tot_en, Y, 
                       method='trust-constr',
                       constraints = [int_cons1, int_cons2],

               options={'maxiter': 3000, 'verbose':0, 'gtol' : 1e-4} 
                      )
        X = res.x
        resX[:,i] = X        
        En_val[i] = tot_en(X)
        
       
        x1, x2, phi_val[i], dt = separation2(X)
        
        dtval[i] = dt
        print("phi = ", phi_val[i])
       
        print("En = ", En_val[i])
        
    j = np.argmin(En_val)
    
    
    print("The best ", phi_val[j], En_val[j], dtval[j])
   
    
    return(2* phi_val[j], En_val[j], dtval[j]*N / 2)
    

